In [3]:
# colab has an older version of beautifulsoup by default
# here we upgrade it
# if you are working on your own computer, you can probably comment this step out and skip it
!pip install --upgrade beautifulsoup4

# New section

In [1]:
# 1. import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [2]:
# 2. find url and store it in a variable
url = "https://en.wikipedia.org/wiki/Berlin"

In [3]:
# 3. download html with a get request
response = requests.get(url)
response.status_code # 200 status code means OK!

200

In [4]:
# 4.1. parse html (create the 'soup')
soup = BeautifulSoup(response.content, "html.parser")
# 4.2. check that the html code looks like it should
#soup

In [5]:
# 5. retrieve/extract the desired info (here, you'll paste the "Selector" you copied before to get the element that belongs to the top movie)

# let's first try to get the name of the city
# by copying the selector we can see that it has the id firstHeading (it also has a class by the same name!)
soup.select("#firstHeading")

[<h1 class="firstHeading mw-first-heading" id="firstHeading"><span class="mw-page-title-main">Berlin</span></h1>]

In [6]:
soup.select("#firstHeading")[0].get_text()

'Berlin'

In [10]:
# Let's use this class, infobox-data, to target the information country
soup.select(".infobox-data")[0].get_text()

'\xa0Germany'

In [27]:
#soup.select(".infobox-data")[0].get_text()

Now we just carry on exploring the html, finding classes, ids, and selectors to target the information we need. Hopefully these classes and selectors will be universal across all cities on wikipedia, but it is likely that they will change in a few places, and we will have to try to make our code robust to this

In [7]:
def recreate_wiki(cities):
  
  list_for_df = []
  
 
  for city in cities:

    url = f'https://en.wikipedia.org/wiki/{city}'


    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')

    response_dict = {}
    response_dict['city'] = soup.select(".firstHeading")[0].get_text()
    response_dict['country'] = soup.select(".infobox-data")[0].get_text()
    response_dict['latitude'] = soup.select(".latitude")[0].get_text()
    response_dict['longitude'] = soup.select(".longitude")[0].get_text()
   
    if soup.select_one('th.infobox-header:-soup-contains("Population")'):
        response_dict['population'] = soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))
    

    list_for_df.append(response_dict)

  cities_df = pd.DataFrame(list_for_df)
  cities_df['latitude'] = cities_df['latitude'].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)
  cities_df['longitude'] = cities_df['longitude'].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)
  cities_df['population'] = cities_df['population'].str.replace(',','', regex = False)

  return cities_df

In [8]:
list_of_cities = ['Berlin', 'Hamburg', 'London', 'Manchester', 'Barcelona']
city = recreate_wiki(list_of_cities)
city

,city,country,latitude,longitude,population
0,Berlin,Germany,52.3112,13.2418,3769495
1,Hamburg,Germany,53.3300,10.0000,1845229
2,London,United Kingdom,51.3026,0.739,8799800
3,Manchester,United Kingdom,53.2846,2.1443,552858
4,Barcelona,Spain,41.23N,2.11E,1620343


In [15]:
cityid = [1,2,3,4,5]
city["cityid"] = cityid

In [16]:
city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   city        5 non-null      object
 1   country     5 non-null      object
 2   latitude    5 non-null      object
 3   longitude   5 non-null      object
 4   population  5 non-null      object
 5   cityid      5 non-null      int64 
dtypes: int64(1), object(5)
memory usage: 368.0+ bytes


In [17]:
schema="e_scooter"   # name of the database you want to use here
host="127.0.0.1"        # to connect to your local server
user="root"
password="saideshmukh16" # your password!!!!
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

print("b")

b


In [18]:
city.to_sql("city_df", if_exists = "append", con = con, index = False)

5

## Alternate to get population:

In [25]:
#df=pd.DataFrame(list(zip(city, country, population, coords)), columns =['city', 'country', 'population', 'coords'])

In [11]:
#here we make our soup for the city
r = requests.get("https://en.wikipedia.org/wiki/Berlin")
soup = BeautifulSoup(r.content, 'html.parser')
soup.select('.infobox-label')

[<th class="infobox-label" scope="row">Country</th>,
 <th class="infobox-label" scope="row"><a href="/wiki/States_of_Germany" title="States of Germany">State</a></th>,
 <th class="infobox-label" scope="row"> • Body</th>,
 <th class="infobox-label" scope="row"> • <a href="/wiki/Governing_Mayor_of_Berlin" title="Governing Mayor of Berlin">Governing Mayor</a></th>,
 <th class="infobox-label" scope="row"> • <a class="mw-redirect" href="/wiki/Bundesrat_of_Germany" title="Bundesrat of Germany">Bundesrat votes</a></th>,
 <th class="infobox-label" scope="row"> • <a href="/wiki/Bundestag" title="Bundestag">Bundestag seats</a></th>,
 <th class="infobox-label" scope="row"> • City/State</th>,
 <th class="infobox-label" scope="row"> • Urban<div class="ib-settlement-fn"></div></th>,
 <th class="infobox-label" scope="row"> • Metro<div class="ib-settlement-fn"></div></th>,
 <th class="infobox-label" scope="row">Elevation<div class="ib-settlement-fn"></div></th>,
 <th class="infobox-label" scope="row">